In [1]:
from zhipuai import ZhipuAI
import json

# 读取配置文件
confige_path = "../confige.json"
with open(confige_path,"r",encoding="utf-8") as f:
    confige = json.load(f)  

sourcefile_path = confige["sourcefile_path"]
aitrans_path = confige["aitrans_path"]
transjson_path = confige["transjson_path"]


FileNotFoundError: [Errno 2] No such file or directory: 'D:/code/python_code/ai/zhipu/confige.json'

通过智谱ai的api将源数据转换成（题目-选项-答案-解析）格式

In [ ]:
client = ZhipuAI(api_key="297dc7f0e8dcbe31a21426081d8e9ee0.7OlrOM13fTTnhWdP")

"""
    已经完成common1.txt
    已经完成common7.txt
    已经完成common6.txt
    已完成common8.txt
    已经完成common9.txt
    已经完成common10.txt
    已经完成common2.txt
    已经完成common11.txt
    已经完成common12.txt
    已经完成common13.txt
    已经完成common14.txt
    已经完成common15.txt
"""

#获取文件行数，用于循环
line_count = 0
with open(sourcefile_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    line_count = len(lines)

#使用智谱ai的api将源数据转换成（题目-选项-答案-解析）格式
for line in range(1, line_count):
    content = "我的要求：不要输出我说的话；格式:(题目:\n选项:\nA. \nB. \nC. \nD. \n答案:\n解析:);每一个选项里都要有内容，解析只要一行，我提供的信息:"
    data = ""
    with open(sourcefile_path, "r", encoding="utf-8") as f:
        data = lines[line-1]
    messages = [{"role":"user","content":content+data}]
    response = client.chat.completions.create(messages= messages,model="glm-4-flash")

    #只录入正确格式的文件
    result = response.choices[0].message.content
    result = result.replace("\n\n","\n")
    question = result.split("\n")[0]
    question_count = len(result.split("\n")[0])
    if result.count("\n")==7 and question.count("题目:")==1 and question_count>5:
        with open(aitrans_path, "a", encoding="utf-8") as f:
            f.write(result+"\n")
                

将文件格式转换成json格式

In [7]:
def convert_to_json(file_path):
    # 初始化一个空列表来存储数据
    data = []

    # 打开文件并逐行读取
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # 初始化一个字典来存储当前问题的数据
    current_question = {}

    # 遍历每一行并处理它们
    for i, line in enumerate(lines):
        line = line.strip()  # 移除行首行尾的空白字符

        # 根据行的位置确定其角色
        if i % 8 == 0:
            current_question['题目'] = line[3:] 
        elif i % 8 == 1:
            current_question['选项'] = {}
        elif i % 8 in [2, 3, 4, 5]:
            option = line[0]  # 提取选项字母（A, B, C, 或 D）
            current_question['选项'][option] = line[2:]  # 存储选项的内容
        elif i % 8 == 6:
            current_question['答案'] = line[3:] 
        elif i % 8 == 7:
            current_question['解析'] = line[3:] 
            # 将当前问题添加到数据列表中并重置字典
            data.append(current_question)
            current_question = {}

    # 将数据列表转换为JSON格式
    json_data = json.dumps(data, ensure_ascii=False, indent=4)
    return json_data

with open(transjson_path, 'w', encoding='utf-8') as output_file:
    output_file.write(convert_to_json(aitrans_path))
